## 4조 과제 - 크롤링 실습 (Beautiful, Selenium 실습)
> 허유진, 윤이정, 오정현, 한수현

### 🐾 과제 설명
지도 제작 전문 회사 (주)커피프린스는 매일 전국 주유소에서 가장 싼 곳을 찾아 사용자에게 제공해주고자 한다.  
회사에서 매일 조사를 할 수 없기 때문에 [오피넷](https://www.opinet.co.kr/)에서 제공해주는 정보를 사용하고자 한다.  
업무를 배정받은 팀 4호점은 한동안 구를 모두 바꾸어 가며 파일을 모두 다운로드 받았다.  
팀원 이정이는 해당 업무의 자동화를 제안하고 싶었고, `서울` 지역을 대상으로 자동화하는 코드를 작성해 팀장을 설득하려고 한다.  
이정이가 목적을 달성할 수 있도록 크롤링 코드를 작성해보자.  

**TO-DO**  

- [x] [오피넷](https://www.opinet.co.kr/)에서 서울 지역 모든 주유소 가격 정보 다운로드 자동화하기
- [x] 오류가 발생할 경우, markdown 형식으로 정리하기

In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By 

### 겪은 문제점 1
**발생 코드**  
```python
from selenium.webdriver.common.by import By 
driver = webdriver.Chrome('chromedriver')
```

**발생 오류**  
Attribute Error : 'str' object has no attribute '_ignore_local_proxy' with ChromeDriverManager

**발생 이유**  
selenium 4.10부터 chromedriver의 절대경로는 Service로 전달해준다.

**해결 방법**  

- chromedriver가 코드와 같은 폴더에 있는 경우

```python
from selenium import webdriver
driver = webdriver.Chrome()
# ...
driver.quit()
```

- chromedriver의 절대경로를 명시해주는 경우

```python
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
service = Service(executable_path="PATH_TO_DRIVER") # pwd 로 chromedriver가 위치한 경로를 확인 후 넣어주어야 함
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)
# ...
driver.quit()
```

### 겪은 문제점 2
**발생 이유**
mac os에서 chromedriver 실행 권한이 없어서 발생하는 문제  

**해결 방법** : chromedriver에 권한 부여
- **reference** : [hanakim120님 [Mac] Chrome driver 설치, Path 설정, mac 에러 해결](https://velog.io/@hanakim120/MacOS-Chrome-driver-%EC%84%A4%EC%B9%98%ED%95%98%EA%B3%A0-Path-%EC%84%A4%EC%A0%95-mac-%EC%97%90%EB%9F%AC-%ED%95%B4%EA%B2%B0)
```bash
xattr -d com.apple.quarantine /usr/local/bin/chromedriver
```


In [2]:
driver = webdriver.Chrome()

In [3]:
driver.get("https://www.opinet.co.kr/searRgSelect.do")
time.sleep(10)

In [5]:
area = driver.find_element(By.XPATH, '//*[@id="SIDO_NM0"]')

In [6]:
area.send_keys('서울')

### 겪은 문제점 3
**문제 사항**  
`gu_list = gu_list_raw.find_elements(By.TAG_NAME, 'option')` 실행 오류  

**발생 원인**  
gu_list_raw 로딩 오류 발생  

```python
gu_list_raw = driver.find_element(By.XPATH, '//*[@id="SIGUNGU_NM0"]')
# find_elements 가 되면 다른 코드가 실행되어 find_elements가 담고 있는 내용이 달라진다.
```
 
**함수 정리**  
> html 등 주어진 문서에서 element를 어떻게 선택할지 전략을 선택하고, 값을 넣어주면 해당하는 elements 를 가져온다.


**find_element(by='id', value: Optional[str] = None) → selenium.webdriver.remote.webelement.WebElement
**  
Find an element given a By strategy and locator.
```bash
Usage:	
element = driver.find_element(By.ID, 'foo')
Return type:	
WebElement

```

**find_elements(by='id', value: Optional[str] = None) → List[selenium.webdriver.remote.webelement.WebElement]**  
Find elements given a By strategy and locator.

```bash
Usage:	
elements = driver.find_elements(By.CLASS_NAME, 'foo')
Return type:	
list of WebElement
```

- **reference** : [selenium.webdriver.common By의 쓰임새](https://www.selenium.dev/selenium/docs/api/py/webdriver/selenium.webdriver.common.by.html)

**By 속성**  
각 전략에 따라서 html 요소를 선택함  
```
CLASS_NAME = 'class name'
CSS_SELECTOR = 'css selector'
ID = 'id'
LINK_TEXT = 'link text'
NAME = 'name'
PARTIAL_LINK_TEXT = 'partial link text'
TAG_NAME = 'tag name'
XPATH = 'xpath'
```


<details>
<summary>▼ By 전략 별 설명보기 클릭</summary>

<!-- summary 아래 한칸 공백 두어야함 -->

`By.ID`: ID 속성 값에 따라 요소를 찾습니다.  
`By.NAME`: name 속성 값에 따라 요소를 찾습니다.  
`By.CLASS_NAME`: class 속성 값에 따라 요소를 찾습니다.  
`By.TAG_NAME`: HTML 태그 이름에 따라 요소를 찾습니다.  
`By.LINK_TEXT`: 정확한 텍스트를 가진 앵커 요소(a 태그)를 찾습니다.   
`By.PARTIAL_LINK_TEXT`: 부분 텍스트를 가진 앵커 요소(a 태그)를 찾습니다.  
`By.CSS_SELECTOR`: CSS 선택자를 사용하여 요소를 찾습니다.  
`By.XPATH`: XPath 표현식을 사용하여 요소를 찾습니다.  
</details>



In [7]:
# 구/데이터 입력을 위한 xpath 확인
gu_list_raw = driver.find_element(By.XPATH, '//*[@id="SIGUNGU_NM0"]')

In [8]:
gu_list = gu_list_raw.find_elements(By.TAG_NAME, 'option')

In [9]:
gu_list

[<selenium.webdriver.remote.webelement.WebElement (session="efc932ef3e1f614afd20d945e0f2541f", element="740AF6CE2AA0E075AF56D47FE12215AE_element_54")>,
 <selenium.webdriver.remote.webelement.WebElement (session="efc932ef3e1f614afd20d945e0f2541f", element="740AF6CE2AA0E075AF56D47FE12215AE_element_55")>,
 <selenium.webdriver.remote.webelement.WebElement (session="efc932ef3e1f614afd20d945e0f2541f", element="740AF6CE2AA0E075AF56D47FE12215AE_element_56")>,
 <selenium.webdriver.remote.webelement.WebElement (session="efc932ef3e1f614afd20d945e0f2541f", element="740AF6CE2AA0E075AF56D47FE12215AE_element_57")>,
 <selenium.webdriver.remote.webelement.WebElement (session="efc932ef3e1f614afd20d945e0f2541f", element="740AF6CE2AA0E075AF56D47FE12215AE_element_58")>,
 <selenium.webdriver.remote.webelement.WebElement (session="efc932ef3e1f614afd20d945e0f2541f", element="740AF6CE2AA0E075AF56D47FE12215AE_element_59")>,
 <selenium.webdriver.remote.webelement.WebElement (session="efc932ef3e1f614afd20d945e0f2

In [10]:
gu_names = [option.get_attribute('value') for option in gu_list]

In [11]:
gu_names.remove('')
gu_names

['강남구',
 '강동구',
 '강북구',
 '강서구',
 '관악구',
 '광진구',
 '구로구',
 '금천구',
 '노원구',
 '도봉구',
 '동대문구',
 '동작구',
 '마포구',
 '서대문구',
 '서초구',
 '성동구',
 '성북구',
 '송파구',
 '양천구',
 '영등포구',
 '용산구',
 '은평구',
 '종로구',
 '중구',
 '중랑구']

In [12]:
element = driver.find_element(By.ID,'SIGUNGU_NM0')
element.send_keys(gu_names[0]) # 지칭은 여기까지

In [13]:
#조회버튼의 Xpath를 찾아서 클릭
xpath ='''//*[@id="searRgSelect"]/span'''
element_sel_gu = driver.find_element(By.XPATH, xpath).click()

In [19]:
import time

# 반복문을 이용하여 엑셀파일 다운로드 진행
for gu in (gu_names):
    element = driver.find_element(By.ID, 'SIGUNGU_NM0')
    element.send_keys(gu)
    time.sleep(2)  # 데이터 획득 위한 지연 시간
    xpath = '''//*[@id="searRgSelect"]/span'''
    element_sel_gu = driver.find_element(By.XPATH, xpath).click()
    time.sleep(1)
    xpath = '''//*[@id="glopopd_excel"]/span'''
    element_get_excel = driver.find_element(By.XPATH, xpath).click()
    time.sleep(1)

## ✨ Congratulations! 